In [1]:
import numpy as np
#import matplotlib as plt
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import glob, os
import random
import tensorflow as tf
from tensorflow import keras



ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
val_target_FILENAME = "/afs/ictp.it/home/j/jahmad/Downloads/val_target.npy"
val_input_FILENAME = "/afs/ictp.it/home/j/jahmad/Downloads/val_input.npy"

In [ ]:
val_input_dataset=np.load(val_input_FILENAME)
val_target_dataset=np.load(val_target_FILENAME)
print(val_input_dataset.shape)


In [ ]:
# temp
temp_surface = val_input_dataset[:,59]
temp_mid = val_input_dataset[:,29]
temp_top = val_input_dataset[:,0]

# specific humidity
sp_humidity_surface = val_input_dataset[:,60+29]
sp_humidity_mid = val_input_dataset[:,60+29]
sp_humidity_top = val_input_dataset[:,60]

# surface pressure
surface_pressure = val_input_dataset[:,-4]

# Insolation
insolation = val_input_dataset[:,-3]

# Surface latent heat flux
surface_latent_heat = val_input_dataset[:,-2]

# Surface sensible heat flux
surface_sensible_heat = val_input_dataset[:,-1]

